# 01 — Connect & EDA (Olist)

This notebook connects to PostgreSQL, runs sanity checks, builds curated tables for BI, and performs RFM.

In [1]:
%pip install -q sqlalchemy psycopg2-binary pandas



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Connect to Postgres using your macOS login
from sqlalchemy import create_engine, text
import getpass

USER = getpass.getuser()      # e.g., "rahimrzayev"
DB   = "olist"
engine = create_engine(f"postgresql+psycopg2://{USER}@127.0.0.1:5432/{DB}", future=True)

with engine.connect() as con:
    print("DB ok:", con.execute(text("select 1")).scalar() == 1)


DB ok: True


In [3]:
from sqlalchemy import text
with engine.connect() as con:
    tables = pd.read_sql(
        text("SELECT table_name FROM information_schema.tables WHERE table_schema='public' ORDER BY 1"), con
    )
display(tables.head(20))

,table_name
0,customers
1,geolocation
2,order_items
3,order_payments
4,order_reviews
5,orders
6,product_category_name_translation
7,products
8,sellers


In [5]:
# Build order-level fact (purchase->delivery timing and GMV)
q = """
WITH oi AS (
  SELECT order_id,
         COUNT(*) AS items_count,
         SUM(price)::numeric(14,2) AS items_value,
         SUM(freight_value)::numeric(14,2) AS freight_value,
         SUM(price + freight_value)::numeric(14,2) AS gmv
  FROM order_items
  GROUP BY 1
)
SELECT 
  o.order_id,
  o.customer_id,
  o.order_status,
  o.order_purchase_timestamp,
  o.order_delivered_customer_date,
  o.order_estimated_delivery_date,
  (o.order_delivered_customer_date IS NOT NULL AND o.order_delivered_customer_date <= o.order_estimated_delivery_date) AS delivered_on_time,
  oi.items_count,
  oi.items_value,
  oi.freight_value,
  oi.gmv
FROM orders o
LEFT JOIN oi USING (order_id);
"""
with engine.connect() as con:
    df_orders = pd.read_sql(q, con, parse_dates=[
        "order_purchase_timestamp", "order_delivered_customer_date", "order_estimated_delivery_date"
    ])
df_orders["order_month"] = df_orders["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()
df_orders["delivery_days"] = (df_orders["order_delivered_customer_date"] - df_orders["order_purchase_timestamp"]).dt.days
df_orders.head()


,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivered_on_time,items_count,items_value,freight_value,gmv,order_month,delivery_days
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29,True,1.0,58.90,13.29,72.19,2017-09-01,7.0
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15,True,1.0,239.90,19.93,259.83,2017-04-01,16.0
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05,True,1.0,199.00,17.87,216.87,2018-01-01,7.0
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20,True,1.0,12.99,12.79,25.78,2018-08-01,6.0
4,00048cc3ae777c65dbb7d2a0634bc1ea,816cbea969fe5b689b39cfc97a506742,delivered,2017-05-15 21:42:34,2017-05-22 13:44:35,2017-06-06,True,1.0,21.90,12.69,34.59,2017-05-01,6.0


In [6]:
# Join customers for unique_id and compute RFM on delivered orders
with engine.connect() as con:
    customers = pd.read_sql("SELECT customer_id, customer_unique_id FROM customers;", con)

df = df_orders.merge(customers, on="customer_id", how="left")

df_delivered = df[df["order_status"]=="delivered"].copy()
df_delivered = df_delivered[df_delivered["order_purchase_timestamp"].notna()]

# RFM
snapshot_date = df_delivered["order_purchase_timestamp"].max()
rfm = (df_delivered
    .groupby("customer_unique_id", as_index=False)
    .agg(
        last_purchase=("order_purchase_timestamp","max"),
        frequency=("order_id","nunique"),
        monetary=("gmv","sum")
    ))
rfm["recency_days"] = (snapshot_date - rfm["last_purchase"]).dt.days

# Quantile-based scoring (1-5)
r_labels = [5,4,3,2,1]  # lower recency days => better
f_labels = m_labels = [1,2,3,4,5]
rfm["R"] = pd.qcut(rfm["recency_days"].rank(method="first"), 5, labels=r_labels)
rfm["F"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=f_labels)
rfm["M"] = pd.qcut(rfm["monetary"].rank(method="first"), 5, labels=m_labels)
rfm["RFM_Score"] = rfm[["R","F","M"]].astype(int).sum(axis=1)

# Simple segment mapping
def segment(row):
    if row.R >= 4 and row.F >= 4 and row.M >= 4: return "Champions"
    if row.R >= 4 and row.F >= 3: return "Loyal"
    if row.R <= 2 and row.F <= 2: return "At Risk"
    if row.F >= 4 and row.M <= 3: return "Potential Loyalist"
    return "Others"
rfm["segment"] = rfm.apply(segment, axis=1)
rfm.head(10)


,customer_unique_id,last_purchase,frequency,monetary,recency_days,R,F,M,RFM_Score,segment
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,111,4,1,4,9,Others
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,114,4,1,1,6,Others
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,536,1,1,2,4,At Risk
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,320,2,1,1,4,At Risk
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,287,2,1,4,7,At Risk
5,0004bd2a26a76fe21f786e4fbd80607f,2018-04-05 19:33:16,1,166.98,145,4,1,4,9,Others
6,00050ab1314c0e55a6ca13cf7181fecf,2018-04-20 12:57:23,1,35.38,131,4,1,1,6,Others
7,00053a61a98854899e70ed204dd4bafe,2018-02-28 11:15:41,1,419.18,182,3,1,5,9,Others
8,0005e1862207bf6ccc02e4228effd9a0,2017-03-04 23:32:12,1,150.12,542,1,1,4,6,At Risk
9,0005ef4cd20d2893f0d9fbd94d3c0d97,2018-03-12 15:22:12,1,129.76,169,4,1,3,8,Others


In [10]:
# Build order-level fact with GMV, delivery_days, on_time, review_score, customer_unique_id
q = """
WITH oi AS (
  SELECT
    order_id,
    COUNT(*)                          AS items_count,
    SUM(price)::numeric(14,2)         AS items_value,
    SUM(freight_value)::numeric(14,2) AS freight_value,
    SUM(price + freight_value)::numeric(14,2) AS gmv
  FROM order_items
  GROUP BY 1
)
SELECT
  o.order_id,
  o.customer_id,
  c.customer_unique_id,
  o.order_status,
  o.order_purchase_timestamp,
  o.order_delivered_customer_date,
  o.order_estimated_delivery_date,
  -- on-time flag
  (o.order_delivered_customer_date IS NOT NULL
   AND o.order_delivered_customer_date <= o.order_estimated_delivery_date) AS delivered_on_time,
  -- amounts
  oi.items_count,
  oi.items_value,
  oi.freight_value,
  oi.gmv,
  -- review
  r.review_score
FROM orders o
LEFT JOIN oi         USING (order_id)
LEFT JOIN customers c ON c.customer_id = o.customer_id
LEFT JOIN order_reviews r ON r.order_id = o.order_id
"""

from sqlalchemy import text
with engine.connect() as con:
    df_orders = pd.read_sql(text(q), con, parse_dates=[
        "order_purchase_timestamp", "order_delivered_customer_date", "order_estimated_delivery_date"
    ])

# extra derived fields
df_orders["order_month"] = df_orders["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()
df_orders["delivery_days"] = (df_orders["order_delivered_customer_date"] - df_orders["order_purchase_timestamp"]).dt.days
df_orders.head()

,order_id,customer_id,customer_unique_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivered_on_time,items_count,items_value,freight_value,gmv,review_score,order_month,delivery_days
0,fc046d7776171871436844218f817d7d,1458a88f3713f047a6d4a23a8ee0f3e5,4ea3dd6fbcdc60a8ac2ee17617df1955,delivered,2018-02-20 18:07:05,2018-03-19 14:34:39,2018-03-19,False,1.0,145.00,15.76,160.76,5.0,2018-02-01,26.0
1,d4665434b01caa9dc3e3e78b3eb3593e,430629c51214778912efab67e023a8f1,3b9b60dd4a5ca201735e213fba2946f1,delivered,2018-06-18 05:22:00,2018-06-21 22:48:57,2018-07-13,True,1.0,60.00,19.61,79.61,5.0,2018-06-01,3.0
2,e28abf2eb2f1fbcbdc2dd0cd9a561671,b1ff51be5960d3ed39bb63679f068555,f7485c007aaec8770e5fda0aad0fa419,delivered,2018-04-17 21:54:15,2018-04-24 18:12:22,2018-05-10,True,1.0,150.00,17.96,167.96,5.0,2018-04-01,6.0
3,04fb47576993a3cb0c12d4b25eab6e4e,c158fb97a12041c9305718081a78b064,df415a4809c4b0d9c76a68dd167a246a,delivered,2017-05-25 12:53:25,2017-05-29 21:22:04,2017-06-07,True,1.0,259.90,41.77,301.67,5.0,2017-05-01,4.0
4,5f358d797a49fe2f24352f73426215f6,e44e2e5c3ad690e075b368557195abb7,15bee0115455cd1b290ab53a541bd858,delivered,2017-08-10 14:01:14,2017-08-11 17:45:52,2017-08-23,True,1.0,68.89,8.72,77.61,5.0,2017-08-01,1.0


In [15]:
# Robust repo root detection (no git CLI)
from pathlib import Path
import os

def find_repo_root(start: Path = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p, *p.parents]:
        if (cand / ".git").exists():
            return cand
    # Fallback to ~/ecommerce-analytics-olist if present
    default = Path.home() / "ecommerce-analytics-olist"
    return default if default.exists() else p

ROOT = find_repo_root()
OUT = ROOT / "data" / "processed"
OUT.mkdir(parents=True, exist_ok=True)
print("Saving to:", OUT)


Saving to: /Users/rahimrzayev/ecommerce-analytics-olist/data/processed


In [19]:
# Write exports into THIS repo under data/processed
from pathlib import Path
import pandas as pd
import os

# Point to your repo's data/processed (hard-set so it doesn't depend on the notebook's cwd)
OUT = Path("/Users/rahimrzayev/ecommerce-analytics-olist/data/processed")
OUT.mkdir(parents=True, exist_ok=True)
print("Saving to:", OUT)

# Monthly KPIs from df (purchase month)
dfm = df.copy()
dfm["month"] = dfm["order_purchase_timestamp"].dt.to_period("M").dt.to_timestamp()

monthly = (
    dfm.groupby("month", as_index=False)
       .agg(
           orders_cnt=("order_id", "nunique"),
           gmv=("gmv", "sum"),
           delivered_total=("order_delivered_customer_date", lambda s: int(s.notna().sum())),
           ontime_deliveries=("delivered_on_time", "sum"),
       )
)

# Ensure numeric and compute rates/rolling/AOV/MoM
monthly["ontime_deliveries"] = pd.to_numeric(monthly["ontime_deliveries"])
monthly["delivered_total"]   = pd.to_numeric(monthly["delivered_total"])

monthly["ontime_rate"] = (monthly["ontime_deliveries"] /
                          monthly["delivered_total"].replace(0, pd.NA)).fillna(0).round(4)
monthly["orders_prev_month"] = monthly["orders_cnt"].shift(1)
monthly["orders_mom_pct"] = ((monthly["orders_cnt"] - monthly["orders_prev_month"]) /
                             monthly["orders_prev_month"]).round(4)
monthly["orders_rolling_3mo_avg"] = monthly["orders_cnt"].rolling(3, min_periods=1).mean().round(2)
monthly["aov"] = (monthly["gmv"].fillna(0) /
                  monthly["orders_cnt"].replace(0, pd.NA)).fillna(0).round(2)

# (Optional) review metrics if your df has review_score
if "review_score" in dfm.columns:
    rev = (
        dfm.groupby("month", as_index=False)
           .agg(
               review_cnt=("review_score", lambda s: int(s.notna().sum())),
               avg_review_score=("review_score", "mean"),
               share_pos_reviews=("review_score",
                                  lambda s: float((s >= 4).sum()) / s.notna().sum() if s.notna().sum() else 0.0),
           )
    )
    rev["avg_review_score"]  = rev["avg_review_score"].fillna(0).round(2)
    rev["share_pos_reviews"] = rev["share_pos_reviews"].fillna(0).round(4)
    monthly = monthly.merge(rev, on="month", how="left").fillna(
        {"review_cnt": 0, "avg_review_score": 0, "share_pos_reviews": 0}
    )

# Save all three exports
(df.to_csv(OUT / "orders_clean.csv", index=False))
(monthly.to_csv(OUT / "monthly_kpis.csv", index=False))
(rfm.to_csv(OUT / "customer_rfm.csv", index=False))

print("Wrote:")
for f in ["monthly_kpis.csv", "orders_clean.csv", "customer_rfm.csv"]:
    p = OUT / f
    print(" ", p, "|", f"{p.stat().st_size:,} bytes")


Saving to: /Users/rahimrzayev/ecommerce-analytics-olist/data/processed


/var/folders/sb/7rcpkmy50fd5d58nhst7fg4h0000gn/T/ipykernel_69569/3675671810.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  monthly["ontime_rate"] = (monthly["ontime_deliveries"] /


Wrote:
  /Users/rahimrzayev/ecommerce-analytics-olist/data/processed/monthly_kpis.csv | 1,816 bytes
  /Users/rahimrzayev/ecommerce-analytics-olist/data/processed/orders_clean.csv | 20,053,853 bytes
  /Users/rahimrzayev/ecommerce-analytics-olist/data/processed/customer_rfm.csv | 7,710,726 bytes
